In [1]:
oc new-project gitea

Now using project "gitea" on server "https://api.crc.testing:6443".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/e2e-test-images/agnhost:2.33 -- /agnhost serve-hostname



In [25]:
oc get scc

NAME                              PRIV    CAPS         SELINUX     RUNASUSER          FSGROUP     SUPGROUP    PRIORITY     READONLYROOTFS   VOLUMES
anyuid                            false   <no value>   MustRunAs   RunAsAny           RunAsAny    RunAsAny    10           false            ["configMap","downwardAPI","emptyDir","persistentVolumeClaim","projected","secret"]
hostaccess                        false   <no value>   MustRunAs   MustRunAsRange     MustRunAs   RunAsAny    <no value>   false            ["configMap","downwardAPI","emptyDir","hostPath","persistentVolumeClaim","projected","secret"]
hostmount-anyuid                  false   <no value>   MustRunAs   RunAsAny           RunAsAny    RunAsAny    <no value>   false            ["configMap","downwardAPI","emptyDir","hostPath","nfs","persistentVolumeClaim","projected","secret"]
hostnetwork                       false   <no value>   MustRunAs   MustRunAsRange     MustRunAs   MustRunAs   <no value>   false            ["configMap"

In [54]:
oc adm policy add-scc-to-user -z default anyuid
oc adm policy add-scc-to-user -z gitea-memcached nonroot

clusterrole.rbac.authorization.k8s.io/system:openshift:scc:anyuid added: "default"


In [71]:
helm uninstall gitea

release "gitea" uninstalled


In [76]:
helm repo add gitea-charts https://dl.gitea.io/charts/
helm install gitea gitea-charts/gitea

"gitea-charts" already exists with the same configuration, skipping
NAME: gitea
LAST DEPLOYED: Thu May  5 10:41:18 2022
NAMESPACE: gitea
STATUS: deployed
REVISION: 1
NOTES:
1. Get the application URL by running these commands:
  echo "Visit http://127.0.0.1:3000 to use your application"
  kubectl --namespace gitea port-forward svc/gitea-http 3000:3000


In [77]:
oc get all

NAME                                   READY   STATUS              RESTARTS   AGE
pod/gitea-0                            0/1     Init:0/3            0          3s
pod/gitea-memcached-66f5c995fd-dv48b   0/1     ContainerCreating   0          3s
pod/gitea-postgresql-0                 0/1     ContainerCreating   0          3s

NAME                                TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)     AGE
service/gitea-http                  ClusterIP   None           <none>        3000/TCP    3s
service/gitea-memcached             ClusterIP   10.217.5.147   <none>        11211/TCP   3s
service/gitea-postgresql            ClusterIP   10.217.4.50    <none>        5432/TCP    3s
service/gitea-postgresql-headless   ClusterIP   None           <none>        5432/TCP    3s
service/gitea-ssh                   ClusterIP   None           <none>        22/TCP      3s

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/gitea-memcached   0/1     1      

In [78]:
oc logs gitea-0  

Error from server (BadRequest): container "gitea" in pod "gitea-0" is waiting to start: PodInitializing


: 1

In [79]:
oc describe pod/gitea-0 

Name:         gitea-0
Namespace:    gitea
Priority:     0
Node:         crc-gh9wd-master-0/192.168.126.11
Start Time:   Thu, 05 May 2022 10:41:19 +0200
Labels:       app=gitea
              app.kubernetes.io/instance=gitea
              app.kubernetes.io/managed-by=Helm
              app.kubernetes.io/name=gitea
              app.kubernetes.io/version=1.16.7
              controller-revision-hash=gitea-85f8f8db8c
              helm.sh/chart=gitea-5.0.7
              statefulset.kubernetes.io/pod-name=gitea-0
              version=1.16.7
Annotations:  checksum/config: d1b2eacb8cf845f80df97f4bee245c38bb9009870c8846f3ccb483dc04aa4b7a
              k8s.v1.cni.cncf.io/network-status:
                [{
                    "name": "openshift-sdn",
                    "interface": "eth0",
                    "ips": [
                        "10.217.0.97"
                    ],
                    "default": true,
                    "dns": {}
                }]
              k8s.v1.cni.cncf.i

In [80]:
oc describe replicaset.apps/gitea-memcached-66f5c995fd

Name:           gitea-memcached-66f5c995fd
Namespace:      gitea
Selector:       app.kubernetes.io/instance=gitea,app.kubernetes.io/name=memcached,pod-template-hash=66f5c995fd
Labels:         app.kubernetes.io/instance=gitea
                app.kubernetes.io/managed-by=Helm
                app.kubernetes.io/name=memcached
                helm.sh/chart=memcached-5.9.0
                pod-template-hash=66f5c995fd
Annotations:    deployment.kubernetes.io/desired-replicas: 1
                deployment.kubernetes.io/max-replicas: 2
                deployment.kubernetes.io/revision: 1
                meta.helm.sh/release-name: gitea
                meta.helm.sh/release-namespace: gitea
Controlled By:  Deployment/gitea-memcached
Replicas:       1 current / 1 desired
Pods Status:    1 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:           app.kubernetes.io/instance=gitea
                    app.kubernetes.io/managed-by=Helm
                    app.kubernetes.io/name=mem

In [81]:
oc get sa

NAME              SECRETS   AGE
builder           2         64m
default           2         64m
deployer          2         64m
gitea-memcached   2         47s


In [82]:
oc get svc

NAME                        TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)     AGE
gitea-http                  ClusterIP   None           <none>        3000/TCP    48s
gitea-memcached             ClusterIP   10.217.5.147   <none>        11211/TCP   48s
gitea-postgresql            ClusterIP   10.217.4.50    <none>        5432/TCP    48s
gitea-postgresql-headless   ClusterIP   None           <none>        5432/TCP    48s
gitea-ssh                   ClusterIP   None           <none>        22/TCP      48s


In [83]:
oc get pvc

NAME                      STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
data-gitea-0              Bound    pvc-d39d9410-d4de-4eac-b78f-e4434b073d2b   10Gi       RWO            nfs-client     64m
data-gitea-postgresql-0   Bound    pvc-f4549e84-1d69-4cb6-8141-21e02415dd49   10Gi       RWO            nfs-client     64m


In [84]:
oc expose service gitea-http 

Error from server (AlreadyExists): routes.route.openshift.io "gitea-http" already exists


: 1

In [85]:
oc get route

NAME         HOST/PORT                           PATH   SERVICES     PORT   TERMINATION   WILDCARD
gitea-http   gitea-http-gitea.apps-crc.testing          gitea-http   http                 None


In [86]:
oc get all

NAME                                   READY   STATUS    RESTARTS   AGE
pod/gitea-0                            1/1     Running   0          68s
pod/gitea-memcached-66f5c995fd-dv48b   1/1     Running   0          68s
pod/gitea-postgresql-0                 1/1     Running   0          68s

NAME                                TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)     AGE
service/gitea-http                  ClusterIP   None           <none>        3000/TCP    68s
service/gitea-memcached             ClusterIP   10.217.5.147   <none>        11211/TCP   68s
service/gitea-postgresql            ClusterIP   10.217.4.50    <none>        5432/TCP    68s
service/gitea-postgresql-headless   ClusterIP   None           <none>        5432/TCP    68s
service/gitea-ssh                   ClusterIP   None           <none>        22/TCP      68s

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/gitea-memcached   1/1     1            1           68s

NAME     

In [88]:
oc exec pod/gitea-0  -- gitea --help

Defaulted container "gitea" out of: gitea, init-directories (init), init-app-ini (init), configure-gitea (init)
NAME:
   Gitea - A painless self-hosted Git service

USAGE:
   gitea [global options] command [command options] [arguments...]

VERSION:
   1.16.7 built with GNU Make 4.3, go1.18.1 : bindata, timetzdata, sqlite, sqlite_unlock_notify

DESCRIPTION:
   By default, gitea will start serving using the webserver with no
arguments - which can alternatively be run by running the subcommand web.

COMMANDS:
   web              Start Gitea web server
   serv             This command should only be called by SSH shell
   hook             Delegate commands to corresponding Git hooks
   dump             Dump Gitea files and database
   cert             Generate self-signed certificate
   admin            Command line interface to perform common administrative operations
   generate         Command line interface for running generators
   migrate          Migrate the database
   keys        

In [89]:
oc exec pod/gitea-0  -- gitea admin --help

Defaulted container "gitea" out of: gitea, init-directories (init), init-app-ini (init), configure-gitea (init)
NAME:
   Gitea admin - Command line interface to perform common administrative operations

USAGE:
   Gitea admin command [command options] [arguments...]

COMMANDS:
   user                Modify users
   repo-sync-releases  Synchronize repository releases with tags
   regenerate          Regenerate specific files
   auth                Modify external auth providers
   sendmail            Send a message to all users

OPTIONS:
   --custom-path value, -C value  Custom path file path (default: "/data/gitea")
   --config value, -c value       Custom configuration file path (default: "/data/gitea/conf/app.ini")
   --version, -v                  print the version
   --work-path value, -w value    Set the gitea working path (default: "/data")
   --help, -h                     show help
   

DEFAULT CONFIGURATION:
     CustomPath:  /data/gitea (GITEA_CUSTOM)
     CustomConf:  /data/g

In [91]:
oc exec pod/gitea-0  -- gitea admin user --help

Defaulted container "gitea" out of: gitea, init-directories (init), init-app-ini (init), configure-gitea (init)
NAME:
   Gitea admin user - Modify users

USAGE:
   Gitea admin user command [command options] [arguments...]

COMMANDS:
   create           Create a new user in database
   list             List users
   change-password  Change a user's password
   delete           Delete specific user by id, name or email

OPTIONS:
   --custom-path value, -C value  Custom path file path (default: "/data/gitea")
   --config value, -c value       Custom configuration file path (default: "/data/gitea/conf/app.ini")
   --version, -v                  print the version
   --work-path value, -w value    Set the gitea working path (default: "/data")
   --help, -h                     show help
   

DEFAULT CONFIGURATION:
     CustomPath:  /data/gitea (GITEA_CUSTOM)
     CustomConf:  /data/gitea/conf/app.ini
     AppPath:     /usr/local/bin/gitea
     AppWorkPath: /data



In [93]:
oc exec gitea-0  -- gitea admin user list

Defaulted container "gitea" out of: gitea, init-directories (init), init-app-ini (init), configure-gitea (init)
2022/05/05 08:45:09 ...s/setting/setting.go:965:loadFromConf() [F] Gitea is not supposed to be run as root. Sorry. If you need to use privileged TCP ports please instead use setcap and the `cap_net_bind_service` permission
command terminated with exit code 1


: 1